In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optimizers

from torchsummary import summary

In [4]:
def normalization(data):
  data_mean = np.mean(data, axis=0)
  data_std  = np.std(data, axis=0)
  data_norm = (data - data_mean) / data_std
  data_norm = np.nan_to_num(data_norm, nan=0) # 欠損値(nan)を0で置換
  del data_mean, data_std
  return data_norm

def indexing(lead_time):
  output_shape = 2
  rt = real_time2[:-lead_time-1]
  t_data = PC_norm[lead_time:]
  print(t_data.shape)
  idx = np.where((rt.year <= 2015))[0]
  t_train = t_data[idx]
  idx = np.where((rt.year > 2015))[0]
  t_test = t_data[idx]
  print('t_train, t_test =', t_train.shape, t_test.shape)
  return rt, t_train, t_test, output_shape

def preprocess(data, rt, lead_time):
  ipt_lag0  = data[10:-lead_time-1]
  ipt_lag5  = data[5:-lead_time-6]
  ipt_lag10 = data[:-lead_time-11]
  # =========
  # 検証データのみ
  idx2 = np.where((rt.year > 2015))[0]
  ipt_lag0_test = ipt_lag0[idx2]
  ipt_lag5_test = ipt_lag5[idx2]
  ipt_lag10_test = ipt_lag10[idx2]
  ipt_test = np.stack([ipt_lag0_test, ipt_lag5_test, ipt_lag10_test], 3)
  return ipt_test

In [6]:
def culc_cor(predict, y_test, lead_time):
  cor = (np.sum(predict[:,0] * y_test[:,0], axis=0) + np.sum(predict[:,1] * y_test[:,1], axis=0)) / \
          (np.sqrt(np.sum(predict[:,0] ** 2 + predict[:,1] ** 2, axis=0)) * np.sqrt(np.sum(y_test[:,0] ** 2 + y_test[:,1] ** 2, axis=0)))
  print('lead time {} day = '.format(lead_time), cor)

In [9]:
if __name__ == '__main__':
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
  #mode = 'mjo'
  mode = 'bsiso'
  data = np.load('/home/maeda/data/bsiso_eeof/prepro_anomaly_8vals.npz')

  olr = data['olr'][80:,24:49,:]
  u850 = data['u850'][80:,24:49,:]
  v850 = data['v850'][80:,24:49,:]
  u200 = data['u200'][80:,24:49,:]
  v200 = data['v200'][80:,24:49,:]
  h850 = data['h850'][80:,24:49,:]
  pr_wtr = data['pr_wtr'][80:,24:49,:]
  sst = data['sst'][80:,24:49,:]

  lat = data['lat'][24:49]
  lon = data['lon']
  time = data['time'][80:]    # 射影後にデータが10日進むため、時刻の方を前進させておく
  real_time = pd.to_datetime(time, unit='h', origin=pd.Timestamp('1800-01-01')) # 時刻をdatetime型に変換

  x = np.stack([olr, u850, v850, u200, v200, h850, pr_wtr, sst], 3)
  x_n = np.zeros(x.shape)
  for i in range(x.shape[3]):
    x_n[:,:,:,i] = normalization(x[:,:,:,i])
  print('x_n = ', x_n.shape)  
  x_test = []

  # bsiso index (eEOF) 読み込み
  if mode == 'bsiso':
    data_file = '/home/maeda/data/bsiso_eeof/bsiso_rt-PCs.npz'
  elif mode == 'mjo':
    data_file = '/home/maeda/data/bsiso_eeof/mjo_rt-PCs.npz'
  PC      = np.load(data_file)['rt_PCs'][:,:2]
  sign    = np.array([-1, 1]).T
  PC_norm = sign * PC / PC.std(axis=0)[np.newaxis,:]
  time2   = np.load(data_file)['time']
  real_time2 = pd.to_datetime(time2, unit='h', origin=pd.Timestamp('1800-01-01')) # 時刻をdatetime型に変換
  print('PCs = ', PC_norm.shape)
  print('time PCs= ', time2.shape)
  print('real time PCs = ', real_time2[0], real_time2[-1])

  def test_step(x, t):
    model.eval()
    preds = model(x)
    loss = loss_fn(preds, t)
    return loss, preds
  

  #lt_box = [0, 5, 10, 15, 20, 25, 30, 35]
  lt_box = np.arange(0, 36, 1)
  seed_box = [0, 7, 7, 9, 5, 1, 2, 1, 1, 8, 
              5, 1, 1, 0, 5, 3, 3, 4, 5, 2,
              3, 3, 3, 5, 5, 6, 6, 7, 2, 6,
              5]
  
  for lead_time in lt_box:
    print('==== lead time : {} day ====='.format(lead_time))
    # answer data
    rt, t_train, t_test, output_shape = indexing(lead_time)
    print('rt, t_train, t_test = ', rt.shape, t_train.shape, t_test.shape)
    # input data
    x_test = []
    for i in range(8):
      _x_test = preprocess(x_n[:,:,:,i], rt, lead_time)
      x_test.append(_x_test)
    x_test = np.stack(np.array(x_test), 3).reshape(-1, 25, 144, 8*3).transpose(0, 3, 1, 2)
    print('x_test =', x_test.shape)
    rt, t_train, t_test, output_shape = indexing(lead_time)
    print('rt, t_train, t_test =', rt.shape, t_train.shape, t_test.shape)
    
    model = torch.load(f'/home/maeda/machine_learning/results/model/kikuchi-8vals_v1/8vals/model_{(lead_time):03}day/seed{(seed_box[lead_time]):03}.pth')
    loss_fn = nn.MSELoss()
    x_test = torch.Tensor(x_test).to(device)
    t_test = torch.Tensor(t_test).to(device)
    test_loss, preds_test = test_step(x_test, t_test)
    print('test loss: {:.3}'.format(
      test_loss.item()
      ))  
 
    predict = preds_test.cpu().detach().numpy()
    t_test  = t_test.cpu().detach().numpy()
    print(predict.shape)
      
    culc_cor(predict, t_test, lead_time)


/tmp/ipykernel_88988/730915222.py:4: RuntimeWarning: invalid value encountered in divide
  data_norm = (data - data_mean) / data_std


x_n =  (15991, 25, 144, 8)
PCs =  (15981, 2)
time PCs=  (15981,)
real time PCs =  1979-04-01 00:00:00 2022-12-31 00:00:00
==== lead time : 0 day =====
(15981, 2)
t_train, t_test = (13424, 2) (2556, 2)
rt, t_train, t_test =  (15980,) (13424, 2) (2556, 2)
x_test = (2556, 24, 25, 144)
(15981, 2)
t_train, t_test = (13424, 2) (2556, 2)
rt, t_train, t_test = (15980,) (13424, 2) (2556, 2)


FileNotFoundError: [Errno 2] No such file or directory: '/home/maeda/machine_learning/results/model/kikuchi-8vals_v1/8vals/model_000day/seed000.pth'